# Generate BitMasks

The purpose of this notebook is to generate bitmasks to be used as filters, using the BAMBOO algorithm.

In [4]:
def generate_bitmaps(
    total_length: str, mask_length: str, mask_characters_list: list
) -> list:
    if total_length % mask_length != 0:
        raise ValueError(f"Total length must be a multiple of {mask_length}.")

    num_bitmaps = total_length // mask_length

    bitmaps = [
        "0" * (i * mask_length)
        + mask_characters_list[0] * (mask_length // 2)
        + mask_characters_list[1] * (mask_length // 2)
        + "0" * ((num_bitmaps - i - 1) * mask_length)
        for i in range(num_bitmaps)
    ]
    return bitmaps

In [5]:
total_length = 1024
mask_length = 8
mask_characters = ["1", "-1"]
bitmaps = generate_bitmaps(total_length, mask_length, mask_characters)
for bitmap in bitmaps:
    print(bitmap)

1111-1-1-1-10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [10]:
len(bitmaps)

128

In [7]:
def generate_specific_bitmask_patterns(L, lengths=[4, 8, 16]):
    filters = []

    def create_specific_patterns(length):
        """Helper function to create specific patterns"""
        half_len = length // 2

        # Pattern 1: First half -1, last half 1
        pattern1 = [-1] * half_len + [1] * (length - half_len)

        # Pattern 2: First half 1, last half -1
        pattern2 = [1] * half_len + [-1] * (length - half_len)

        # Pattern 3: All 1
        pattern3 = [1] * length

        # Pattern 4: All -1
        pattern4 = [-1] * length

        # Pattern 5: Alternating -1 and 1
        pattern5 = [(-1) ** i for i in range(length)]

        # Pattern 6: Alternating 1 and -1
        pattern6 = [1 if i % 2 == 0 else -1 for i in range(length)]

        return [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6]

    for length in lengths:
        num_windows = L // length  # Number of full windows we can fit in L

        for i in range(num_windows):
            # Create a base mask of 0s with length `L`
            base_mask = [0] * L

            # Define the position of the window
            start_pos = i * length
            end_pos = start_pos + length

            # Get all specific patterns for the current window length
            patterns = create_specific_patterns(length)

            for pattern in patterns:
                # Create a mask with the current pattern
                mask = base_mask[:]

                for j in range(length):
                    if start_pos + j < L:
                        mask[start_pos + j] = pattern[j]

                # Convert the mask list to a string
                filters.append("".join(map(str, mask)))

    return filters


# Example usage:
L = 1024
bitmask_filters = generate_specific_bitmask_patterns(L)
for f in bitmask_filters:
    print(f)

-1-1110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [8]:
len(bitmask_filters)

2688

In [11]:
type(bitmask_filters)

list

In [14]:
import csv

In [12]:
def save_patterns_to_csv(patterns, filename):
    with open(filename, "w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        for pattern in patterns:
            csvwriter.writerow([pattern])

In [15]:
save_patterns_to_csv(
    bitmask_filters, "../../data/filters/bitmask_patterns_sliding_window.csv"
)

In [16]:
import numpy as np

In [49]:
data = np.array(
    [
        ["0", "0", "0", "1", "0", "0", "0"],
        ["0", "0", "0", "1", "1", "1", "0"],
        ["0", "0", "0", "1", "0", "1", "0"],
        ["0", "0", "0", "1", "0", "0", "0"],
        ["0", "0", "0", "0", "1", "0", "1"],
    ]
)

In [50]:
filter = np.array(["-1", "1", "-1", "1", "-1", "-1", "-1"])

In [58]:
len(data)

5

In [57]:
len(filter)

7

In [51]:
filtered = np.multiply(data.astype(int), filter.astype(int))

In [52]:
filtered

array([[ 0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  1, -1, -1,  0],
       [ 0,  0,  0,  1,  0, -1,  0],
       [ 0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0, -1]])

In [53]:
# Sum the rows of filtered
row_sums = np.sum(filtered, axis=1)

# Create a column vector with the results
column_vector = np.reshape(row_sums, (-1, 1))

In [54]:
row_sums

array([ 1, -1,  0,  1, -2])

In [55]:
column_vector

array([[ 1],
       [-1],
       [ 0],
       [ 1],
       [-2]])

In [56]:
a = "-1-111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000"